#### 一. 购物评价情感分析
1. 本数据集包含两万多条中文标注语料，涉及六个领域的评论数据  
 对这些评论数据先分词, 再倒入与训练好的词向量, 构建RNN模型进行情感分析
<img src='img/zhongwenpingluing.jpg' height='60%' width='60%'>

#### 二. 数据预处理
1. 将文本数据分词, 并构建标签   
 形成ndaray: x=分词后的语句 , y=0/1
2. 划分训练集和测试机

In [1]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

In [2]:
# 1,记载数据文件
import pandas as pd
neg_df = pd.read_csv('data/neg.csv',header=None,index_col=None)
pos_df = pd.read_csv('data/pos.csv',header=None,index_col=None)

In [3]:
# 原始的预料库负例
neg_df.head()

,0
0,做为一本声名在外的流行书，说的还是广州的外企，按道理应该和我的生存环境差不多啊。但是一看之下...
1,作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...
2,作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...
3,作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。
4,作者的文笔一般，观点也是和市面上的同类书大同小异，不推荐读者购买。


In [4]:
# 原始语料库正例
pos_df.head()

,0
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...


In [5]:
# 对dataframe增加分此后的列
import jieba
cw = lambda x: list(jieba.cut(x))  # jieba.cut返回generator
stpword_path = 'data/stop_words.txt'
# 生成utf-8的停用词表
with open(stpword_path,'rb') as f:
    content = f.read()
    # 按行切分停用词表
    stpword_list = content.splitlines()
    stpword_list = [word.decode('GBK') for word in stpword_list]
    
def cw2(x):
    generator = jieba.cut(x)
    return [word for word in generator if word not in stpword_list]

pos_df['cut_words'] = pos_df[0].apply(cw2)
neg_df['cut_words'] = pos_df[0].apply(cw2)
pos_df.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.641 seconds.
Prefix dict has been built succesfully.


,0,cut_words
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...,"[做, 父母, 一定, 刘墉, 心态, 不断, 学习, 不断, 进步, 不断, 补充, 新鲜..."
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,"[作者, 真有, 英国人, 严谨, 风格, 提出, 观点, 进行, 论述, 论证, 物理学,..."
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,"[作者, 长篇大论, 借用, 详细, 报告, 数据处理, 工作, 计算结果, 支持, 其新,..."
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...,"[作者, 战, 之前, ＂, 拥抱, ＂, 令人, 叫绝, ．, 日本, 战败, 美军, 占..."
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...,"[作者, 少年, 时即, 喜, 阅读, 看出, 精读, 无数, 经典, 一个, 庞大, 内心..."


In [6]:
import numpy as np
# 构建语料库ndarray
x = np.concatenate((pos_df['cut_words'],neg_df['cut_words']))
# 构建标签ndarray:1为正标签,0为负标签
y = np.concatenate((np.ones(len(pos_df)),np.zeros(len(neg_df))),axis = 0)

sentences_num = x.shape[0]
print(sentences_num,'sentences in corpus')

21105 sentences in corpus


In [7]:
# shuffle data
indices = np.arange(sentences_num)
np.random.seed(1)
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

#### 三. 导入训练的词嵌入, 转换文本corpus为index corpus
1. 预训练的词嵌入下载
使用百度文库上, Skip-Gram with Negative Sampling (SGNS)训练的词向量. 文本1.7G,共635974词,300维  
[https://github.com/Embedding/Chinese-Word-Vectors](https://github.com/Embedding/Chinese-Word-Vectors)
2. 

In [8]:
embedding_dir = '/home/lj/data/Deelearning_with_python/sgns.baidubaike.bigram-char'
embedding_dict = {}
from itertools import islice
with open(embedding_dir,'r') as f:
    for line_num,line in enumerate(f): #滤出文件第0行
        if line_num>60000:
            break
        line = line.strip('\n')
        content = line.split(' ')
        word = content[0]
        coefs = np.array(content[1:301],dtype='float32')
        word_num = line_num+1 # 词典编号从1开始,0号预留出来
        embedding_dict[word] = (word_num,coefs)
print('found %s words in embedding_dict'%len(embedding_dict))

found 60001 words in embedding_dict


In [9]:
# 查看形成的词嵌入词典中, '的'字符对应的值
# (词典中的编号, 词向量)
embedding_dict['的']

(2, array([ 5.045200e-01, -1.705800e-01,  2.638530e-01,  5.306370e-01,
         1.101530e-01, -2.658490e-01, -1.900760e-01,  7.454000e-03,
         4.651710e-01,  7.582930e-01, -2.205990e-01, -4.840300e-02,
        -2.912600e-01,  2.067500e-01, -5.822560e-01,  3.393050e-01,
         1.283190e-01,  3.119600e-02, -3.328000e-03,  2.110470e-01,
        -3.468420e-01,  4.714000e-03, -2.035070e-01,  6.305240e-01,
         6.505300e-01, -1.291500e-02, -1.219030e-01,  1.085650e-01,
        -7.396940e-01,  2.838400e-01,  5.618630e-01, -1.761840e-01,
         7.858510e-01, -3.989040e-01,  1.630060e-01,  1.085489e+00,
        -1.393830e-01,  2.006670e-01, -5.734050e-01,  9.604600e-01,
         1.400230e-01, -1.248930e-01, -3.209100e-02, -4.257840e-01,
         3.986000e-02, -4.035770e-01, -3.634310e-01,  3.756700e-01,
        -9.476200e-02, -4.044900e-02,  8.362600e-02,  3.855650e-01,
         1.052700e-01, -2.576370e-01, -3.927490e-01,  9.302000e-03,
        -9.103500e-02, -1.662470e-01, -3.3320

In [10]:
# 将文本型的x, 转换成index型的x
fun1 = lambda xi:[embedding_dict[w][0] for w in xi if w in embedding_dict]
x = np.array([fun1(xi) for xi in x])
x[:2]

array([list([807, 15983, 7397, 1013, 990, 7864, 4601, 2729, 5500, 281, 18479, 1380, 19969, 14549, 17894, 2957, 2238, 1729, 31149, 1679, 1363, 6724, 507, 458, 11621, 11479, 17991, 28, 4366, 19109, 784, 1722, 3316, 20969, 25188, 86, 19109, 1351, 686, 426, 2122, 3713, 35353, 210, 158, 830]),
       list([5486, 4757, 619, 1645, 26856, 4208, 4208, 562, 4720, 4208, 1288, 462, 1631, 370, 5486, 370, 12231, 5486, 459, 49711, 13214, 47773, 1075, 807, 57981, 870, 15885, 53666])],
      dtype=object)

In [11]:
# 划分训练集,测试集
train_samples = int(0.8*sentences_num)
x_train = x[:train_samples]
y_train = y[:train_samples]
x_test = x[train_samples:]
y_test = y[train_samples:]

In [12]:
# 转换embedding_dict为embedding_matrix
n = 300  
m = len(embedding_dict.keys()) + 1 # 0号index没有单词,只是占位符
embedding_matrix = np.zeros((m,n))

for item in embedding_dict.items():
    val = item[1] # (index,embedding_vector)
    embedding_matrix[val[0]] = val[1]

#### 三. 构建神经网络
1. 把评论切成统一长度
2. 后期优化  
 1. 使用gensim自己训练词向量  
 2. 使用fasttext词向量
 3. 验证文字下x和y的匹配, 验证文字下x和index下x的匹配

In [13]:
# 构建神经网络
from keras.models import Sequential
from keras.layers import Embedding,Flatten,LSTM,Dense
from keras import preprocessing
from keras.layers.core import Dropout

# sentence length
maxlen = 100
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test,maxlen=maxlen)

/devkit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
print(embedding_matrix.shape)
print(x_train.shape)
print(y_train.shape)
print(x_train[0])
print(m)
print(n)
print(maxlen)

(60002, 300)
(16884, 100)
(16884,)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   807 15983  7397  1013   990  7864
  4601  2729  5500   281 18479  1380 19969 14549 17894  2957  2238  1729
 31149  1679  1363  6724   507   458 11621 11479 17991    28  4366 19109
   784  1722  3316 20969 25188    86 19109  1351   686   426  2122  3713
 35353   210   158   830]
60002
300
100


In [16]:
model = Sequential()
model.add(Embedding(m,n,input_length=maxlen))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(64))
# model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.set_weights([embedding_matrix])
# model.layers[0].trainable = False  # 冻结Embedding层

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          18000600  
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 128)          219648    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 18,269,721
Trainable params: 18,269,721
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',metrics=['accuracy'])

history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.1)

Train on 15195 samples, validate on 1689 samples
Epoch 1/10
15195/15195 [==============================] - 139s 9ms/step - loss: 0.6986 - acc: 0.5019 - val_loss: 0.6935 - val_acc: 0.4991
Epoch 2/10
15195/15195 [==============================] - 137s 9ms/step - loss: 0.6950 - acc: 0.5046 - val_loss: 0.6948 - val_acc: 0.4985
Epoch 3/10
15195/15195 [==============================] - 138s 9ms/step - loss: 0.6942 - acc: 0.5025 - val_loss: 0.6936 - val_acc: 0.4731
Epoch 4/10
15195/15195 [==============================] - 138s 9ms/step - loss: 0.6934 - acc: 0.5029 - val_loss: 0.6946 - val_acc: 0.4926
Epoch 5/10
15195/15195 [==============================] - 144s 9ms/step - loss: 0.6934 - acc: 0.5017 - val_loss: 0.6983 - val_acc: 0.4352
Epoch 6/10
15195/15195 [==============================] - 326s 21ms/step - loss: 0.6923 - acc: 0.5166 - val_loss: 0.7029 - val_acc: 0.4091
Epoch 7/10
15195/15195 [==============================] - 179s 12ms/step - loss: 0.6904 - acc: 0.5258 - val_loss: 0.7074 -

In [19]:
score = model.evaluate(x_test, y_test)
print ('Test score:', score)

4221/4221 [==============================] - 28s 7ms/step
Test score: [0.7713313046379469, 0.2961383558610298]
